In [9]:
from src.simulators.WF_sim import simulate_waveforms
import sys,os
sys.path.insert(0,'/Users/mxd6118/Desktop/DiffSim')
import pandas as pd
import pickle
from jax import random, device_put
import time
from jax.example_libraries import stax
from jax import jit,vmap, grad
import numpy as np
import jax.numpy as jnp
import jax
import sklearn
from jax.example_libraries import optimizers as jax_opt

import matplotlib.pyplot as plt

key = random.PRNGKey(int(time.time()))
        
key, subkey = random.split(key)

In [10]:
def build_dataloader(number_of_events):
    sys.path.insert(0,'/Users/mxd6118/Desktop/DiffSim/src/utils')
    from dataloaders.krypton_DATES_CUSTOM_DROPOUT import krypton
    # Load the sipm database:
    sipm_db = pd.read_pickle("/Users/mxd6118/Desktop/DiffSim/database/new_sipm.pkl")

    dl = krypton(
        batch_size  = number_of_events,
        db          = sipm_db,
        path        = "/Users/mxd6118/Desktop/DiffSim/kdst",
        run         = 8530,
        shuffle = True,
        drop = 0,
        z_slice = 0,
        )

    return dl

def load_state(file):
    with open(file,"rb") as f:
        params = pickle.load(f)
    return params
    
params_dir = '/Users/mxd6118/Desktop/GAN/src/simulators/'
params_path = os.path.join(params_dir ,"trained_params.pickle")


In [11]:
def init_params(sim_params,subkey):
    
    dis_init, dis_apply = stax.serial(
        stax.Flatten,
        stax.Dense(128),stax.Sigmoid,
        stax.Dense(16), stax.Sigmoid,
        stax.Dense(2),stax.Softmax
    )


    dis_out_size, dis_network_params = dis_init(subkey,(1,47,47))

    parameters = {
    'D_parameters': dis_network_params,
    'S_parameters': sim_params,
    }

    return parameters, dis_apply


def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-8 # Small value to avoid division by zero
    y_pred = jnp.clip(y_pred, epsilon, 1.0 - epsilon)  # Clip values to prevent NaNs
    loss = -(y_true * jnp.log(y_pred) + (1 - y_true) * jnp.log(1 - y_pred))
    return jnp.mean(loss)

In [12]:
@jit
def gen_noise_constant(sipm_waveforms,subkey):

    noise_constant = random.uniform(subkey,shape = sipm_waveforms.shape,minval = -0.5, maxval = 1)
    #noise_constant = random.poisson(subkey,shape = sipm_waveforms.shape,lam = 0.05)
    #noise_constant = random.beta(subkey,shape = sipm_waveforms.shape,a = 2, b = 5)

    return noise_constant
xy_gen_noise_constant = jit(vmap(gen_noise_constant,in_axes=[0,None]))
event_gen_noise_constant = jit(vmap(xy_gen_noise_constant,in_axes=[0,None]))
batch_gen_noise_constant = jit(vmap(event_gen_noise_constant,in_axes=[0,None]))


In [13]:
dataloader = build_dataloader(10)
batch = next(dataloader.iterate())

sim_params = load_state(params_path)

parameters, dis_apply = init_params(sim_params,subkey)

In [140]:
import matplotlib.pyplot as plt
import sklearn

@jit
def forward_pass(batch, parameters,key):
    
    noise = batch_gen_noise_constant(batch['S2Si'],subkey)

    simulated_pmts, simulated_sipms = simulate_waveforms(batch['energy_deposits'],
                                                         parameters['S_parameters'], noise, key)

    
    
    ### try to fix the number of pics per event ###
    
    reshaped_arr = simulated_sipms.reshape(simulated_sipms.shape[0] * 550, 47, 47)

    # Now you have the desired shape, and you can perform matmul if needed
    # For example, let's create a random matrix B of shape (47, 47)
    B = jnp.ones((47, 47))

    # Perform matmul
    result = jnp.matmul(reshaped_arr, B)
    
    
    
    print(result.shape)
    
    #fake_labels = dis_apply(parameters['D_parameters'],filtered_full_batch['Train'])

    #print(f'Fake labels {fake_labels.shape}',flush = True)
    #print(f'Real labels {batch["Labels"].shape}',flush = True)

    #loss_dis = binary_cross_entropy(filtered_full_batch['Labels'],fake_labels)

    return #loss_dis



gradient_fn = jit(jax.value_and_grad(forward_pass, argnums=1,has_aux=False))

opt_init, opt_update, get_params = jax_opt.adamax(1e-3)

opt_state = opt_init(parameters)

    
    
    

In [141]:
for c in range(0,10):

    metrics = {}

    parameters = get_params(opt_state)

    #print(self.parameters,flush = True)

    key = random.PRNGKey(int(time.time()))

    key, subkey = jax.random.split(key)

    loss, gradients = gradient_fn(batch, parameters, subkey)

    #print(gradients['Dis_parameters'])

    opt_state = opt_update(c, gradients, opt_state)

    # # Combine losses

    metrics['loss/loss'] = loss

    metrics.update(parameters)


(5500, 47, 47)


TypeError: Gradient only defined for scalar-output functions. Output was None.

In [199]:
simulated_pmts, simulated_sipms = simulate_waveforms(batch['energy_deposits'],
                                                         parameters['S_parameters'], noise, key)

    
    
### try to fix the number of pics per event ###
test =[]

flatten = simulated_sipms.reshape(10,2209,550)

jnp.sum(flatten,axis = 2).shape

#for i in range(0,result.shape[0]):
#   plt.imshow(result[0])
#   plt.show()

(10, 2209)